In [16]:
import numpy as np
import pandas as pd
import scipy as sp

In [18]:
import pkg_resources
import sys

print("python ==" + str(sys.version_info) +
      "\nnumpy == " + pkg_resources.get_distribution("numpy").version +
      "\npandas == " + pkg_resources.get_distribution("pandas").version +
      "\nscipy == " + pkg_resources.get_distribution("scipy").version)


python ==sys.version_info(major=3, minor=5, micro=5, releaselevel='final', serial=0)
numpy == 1.16.2
pandas == 0.24.2
scipy == 1.2.1
